In [309]:
import numpy as np
import pandas as pd
import tensorflow as tf
import math
import joblib

from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.utils import to_categorical

In [310]:
df = pd.read_csv('face_dataset.csv')
df.head()

,face_token,face_rectangle.top,face_rectangle.left,face_rectangle.width,face_rectangle.height,landmark.contour_chin.x,landmark.contour_chin.y,landmark.contour_left1.x,landmark.contour_left1.y,landmark.contour_left10.x,...,attributes.mouthstatus.other_occlusion,attributes.mouthstatus.close,attributes.mouthstatus.open,attributes.skinstatus.health,attributes.skinstatus.stain,attributes.skinstatus.dark_circle,attributes.skinstatus.acne,attributes.glass.value,image,face_shape
0,8a6e9620dd3a08d9de611c0ec8e36f89,94,65,137,137,116,227,71,103,76,...,0.000,0.001,99.999,12.245,22.748,4.378,7.220,NaN,heart0.jpg,heart
1,ce43bd143ebef3b7908085bdac13367a,168,83,157,157,161,326,79,195,102,...,0.007,0.000,99.993,77.757,5.556,1.683,0.950,NaN,heart1.jpg,heart
2,a45dfa0715c52aaf98dcbb244e08998c,299,198,246,246,335,536,206,330,232,...,0.002,99.939,0.060,54.633,6.944,2.790,1.921,NaN,heart2.jpg,heart
3,4257a9258a394e5853182a88748d492a,275,138,266,266,271,531,137,312,168,...,0.000,0.000,100.000,41.603,3.768,4.404,0.192,NaN,heart3.jpg,heart
4,4534c8426ec65e0610e2c95dcc2e6fca,117,134,144,144,201,256,134,133,147,...,0.000,99.672,0.328,2.445,9.469,10.786,2.185,NaN,heart4.jpg,heart


In [311]:
columns_list = [
    'face_rectangle.top',
    'face_rectangle.width', 'face_rectangle.height',
    'landmark.contour_chin.x', 'landmark.contour_chin.y',
    'landmark.contour_left1.x', 'landmark.contour_left1.y',
    'landmark.contour_left10.x', 'landmark.contour_left10.y',
    'landmark.contour_left11.x', 'landmark.contour_left11.y',
    'landmark.contour_left12.x', 'landmark.contour_left12.y',
    'landmark.contour_left13.x', 'landmark.contour_left13.y',
    'landmark.contour_left14.x', 'landmark.contour_left14.y',
    'landmark.contour_left15.x', 'landmark.contour_left15.y',
    'landmark.contour_left16.x', 'landmark.contour_left16.y',
    'landmark.contour_left2.x', 'landmark.contour_left2.y',
    'landmark.contour_left3.x', 'landmark.contour_left3.y',
    'landmark.contour_left4.x', 'landmark.contour_left4.y',
    'landmark.contour_left5.x', 'landmark.contour_left5.y',
    'landmark.contour_left6.x', 'landmark.contour_left6.y',
    'landmark.contour_left7.x', 'landmark.contour_left7.y',
    'landmark.contour_left8.x', 'landmark.contour_left8.y',
    'landmark.contour_left9.x', 'landmark.contour_left9.y',
    'landmark.contour_right1.x', 'landmark.contour_right1.y',
    'landmark.contour_right10.x', 'landmark.contour_right10.y',
    'landmark.contour_right11.x', 'landmark.contour_right11.y',
    'landmark.contour_right12.x', 'landmark.contour_right12.y',
    'landmark.contour_right13.x', 'landmark.contour_right13.y',
    'landmark.contour_right14.x', 'landmark.contour_right14.y',
    'landmark.contour_right15.x', 'landmark.contour_right15.y',
    'landmark.contour_right16.x', 'landmark.contour_right16.y',
    'landmark.contour_right2.x', 'landmark.contour_right2.y',
    'landmark.contour_right3.x', 'landmark.contour_right3.y',
    'landmark.contour_right4.x', 'landmark.contour_right4.y',
    'landmark.contour_right5.x', 'landmark.contour_right5.y',
    'landmark.contour_right6.x', 'landmark.contour_right6.y',
    'landmark.contour_right7.x', 'landmark.contour_right7.y',
    'landmark.contour_right8.x', 'landmark.contour_right8.y',
    'landmark.contour_right9.x', 'landmark.contour_right9.y',
    'landmark.left_eyebrow_upper_middle.y',
    'landmark.right_eyebrow_upper_middle.y',
    'face_shape'
]


In [312]:
df = df[columns_list]
df.head()

,face_rectangle.top,face_rectangle.width,face_rectangle.height,landmark.contour_chin.x,landmark.contour_chin.y,landmark.contour_left1.x,landmark.contour_left1.y,landmark.contour_left10.x,landmark.contour_left10.y,landmark.contour_left11.x,...,landmark.contour_right6.y,landmark.contour_right7.x,landmark.contour_right7.y,landmark.contour_right8.x,landmark.contour_right8.y,landmark.contour_right9.x,landmark.contour_right9.y,landmark.left_eyebrow_upper_middle.y,landmark.right_eyebrow_upper_middle.y,face_shape
0,94,137,137,116,227,71,103,76,180,80,...,165,194,175,190,184,185,192,93,100,heart
1,168,157,157,161,326,79,195,102,283,109,...,230,234,241,232,253,229,264,177,164,heart
2,299,246,246,335,536,206,330,232,473,243,...,420,433,433,426,446,418,459,307,318,heart
3,275,266,266,271,531,137,312,168,464,179,...,397,391,414,387,431,380,447,287,286,heart
4,117,144,144,201,256,134,133,147,217,152,...,186,269,195,266,204,262,212,122,124,heart


In [313]:
df['head_width'] = df['face_rectangle.width']
df['head_length'] = df['face_rectangle.height']

df = df.drop(columns=['face_rectangle.width', 'face_rectangle.height'])

In [314]:
df['forehead'] = abs(((df['landmark.left_eyebrow_upper_middle.y'] - df['face_rectangle.top']) + (df['landmark.left_eyebrow_upper_middle.y'] - df['face_rectangle.top'])) / 2) / df['head_length']

df = df.drop(columns=['landmark.left_eyebrow_upper_middle.y', 'landmark.right_eyebrow_upper_middle.y'])

In [315]:
new_columns = {}

for i in range(1, 17):
    for j in range(1, 17):
        if (i < j):
            new_columns[f'distance_l{i}l{j}'] = (np.sqrt((df[f'landmark.contour_left{i}.y'] - df[f'landmark.contour_left{j}.y']) ** 2 + (df[f'landmark.contour_left{i}.x'] - df[f'landmark.contour_left{j}.x']) ** 2)) / df['head_length']
            new_columns[f'distance_r{i}r{j}'] = (np.sqrt((df[f'landmark.contour_right{i}.y'] - df[f'landmark.contour_right{j}.y']) ** 2 + (df[f'landmark.contour_right{i}.x'] - df[f'landmark.contour_right{j}.x']) ** 2)) / df['head_length']
            
        new_columns[f'distance_l{i}r{j}'] = (np.sqrt((df[f'landmark.contour_left{i}.y'] - df[f'landmark.contour_right{j}.y']) ** 2 + (df[f'landmark.contour_left{i}.x'] - df[f'landmark.contour_right{j}.x']) ** 2)) / df['head_length']

    new_columns[f'face_width{i}'] = abs(df[f'landmark.contour_right{i}.x'] - df[f'landmark.contour_left{i}.x']) / df['head_width']

    new_columns[f'face_length_left{i}'] = abs(df[f'landmark.contour_left{i}.y'] - df['face_rectangle.top']) / df['head_length']
    new_columns[f'face_length_right{i}'] = abs(df[f'landmark.contour_right{i}.y'] - df['face_rectangle.top']) / df['head_length']

new_df = pd.DataFrame(new_columns)
df = pd.concat([df, new_df], axis=1)

In [316]:
for i in range(1, 17):
    df = df.drop(columns=[f'landmark.contour_right{i}.x', f'landmark.contour_left{i}.x'])
    df = df.drop(columns=[f'landmark.contour_right{i}.y', f'landmark.contour_left{i}.y'])

In [317]:
df = df.drop(columns=['face_rectangle.top', 'landmark.contour_chin.x', 'landmark.contour_chin.y', 'head_width', 'head_length'])
df.head()

,face_shape,forehead,distance_l1r1,distance_l1l2,distance_r1r2,distance_l1r2,distance_l1l3,distance_r1r3,distance_l1r3,distance_l1l4,...,distance_l16r10,distance_l16r11,distance_l16r12,distance_l16r13,distance_l16r14,distance_l16r15,distance_l16r16,face_width16,face_length_left16,face_length_right16
0,heart,0.007299,0.999547,0.066098,0.073357,1.003510,0.124943,0.140404,1.003855,0.191040,...,0.553976,0.490411,0.423735,0.360336,0.292062,0.227334,0.150478,0.145985,0.941606,0.978102
1,heart,0.057325,0.964732,0.063694,0.078785,0.976207,0.133758,0.148696,0.980974,0.197555,...,0.579443,0.513520,0.447223,0.379716,0.306263,0.239849,0.159363,0.159236,0.993631,0.987261
2,heart,0.032520,0.994873,0.065547,0.057056,0.996872,0.130652,0.118166,1.002904,0.199560,...,0.462272,0.410650,0.352980,0.301828,0.246932,0.195334,0.142509,0.142276,0.959350,0.951220
3,heart,0.045113,0.992510,0.067774,0.067774,0.998055,0.135391,0.135338,1.000678,0.199248,...,0.519967,0.464054,0.402835,0.341195,0.282656,0.220109,0.154181,0.154135,0.954887,0.951128
4,heart,0.034722,1.000603,0.069444,0.069444,1.005411,0.138889,0.139062,1.008117,0.201389,...,0.524938,0.463252,0.402358,0.342608,0.277951,0.217062,0.145833,0.145833,0.958333,0.958333


In [318]:
print(df.columns.tolist())


['face_shape', 'forehead', 'distance_l1r1', 'distance_l1l2', 'distance_r1r2', 'distance_l1r2', 'distance_l1l3', 'distance_r1r3', 'distance_l1r3', 'distance_l1l4', 'distance_r1r4', 'distance_l1r4', 'distance_l1l5', 'distance_r1r5', 'distance_l1r5', 'distance_l1l6', 'distance_r1r6', 'distance_l1r6', 'distance_l1l7', 'distance_r1r7', 'distance_l1r7', 'distance_l1l8', 'distance_r1r8', 'distance_l1r8', 'distance_l1l9', 'distance_r1r9', 'distance_l1r9', 'distance_l1l10', 'distance_r1r10', 'distance_l1r10', 'distance_l1l11', 'distance_r1r11', 'distance_l1r11', 'distance_l1l12', 'distance_r1r12', 'distance_l1r12', 'distance_l1l13', 'distance_r1r13', 'distance_l1r13', 'distance_l1l14', 'distance_r1r14', 'distance_l1r14', 'distance_l1l15', 'distance_r1r15', 'distance_l1r15', 'distance_l1l16', 'distance_r1r16', 'distance_l1r16', 'face_width1', 'face_length_left1', 'face_length_right1', 'distance_l2r1', 'distance_l2r2', 'distance_l2l3', 'distance_r2r3', 'distance_l2r3', 'distance_l2l4', 'distance_

In [319]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1006 entries, 0 to 1005
Columns: 546 entries, face_shape to face_length_right16
dtypes: float64(545), object(1)
memory usage: 4.2+ MB


In [320]:
unique_values = df['face_shape'].unique()
print(unique_values)

['heart' 'oblong' 'oval' 'round' 'square']


In [321]:
X = df.drop(columns=['face_shape'])
y = df['face_shape']

In [322]:
df = shuffle(df, random_state=42)

In [323]:
def rf(X, y):    
    le = LabelEncoder()
    y = le.fit_transform(y)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
    rf_classifier.fit(X_train, y_train)
    y_pred = rf_classifier.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)

    print(f"Accuracy: {accuracy:.2f}")
    print("Classification Report:")
    print(report)


In [324]:
rf(X, y)

Accuracy: 0.50
Classification Report:
              precision    recall  f1-score   support

           0       0.40      0.48      0.44        33
           1       0.76      0.57      0.65        49
           2       0.44      0.44      0.44        45
           3       0.39      0.46      0.42        35
           4       0.51      0.50      0.51        40

    accuracy                           0.50       202
   macro avg       0.50      0.49      0.49       202
weighted avg       0.52      0.50      0.50       202



In [325]:
def dnn(X, y):
    le = LabelEncoder()
    y = le.fit_transform(y)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    model = Sequential([
        Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
        Dense(64, activation='relu'),
        Dense(32, activation='relu'),
        Dense(len(np.unique(y)), activation='softmax')  # Adjust the number of units based on the number of classes
    ])

    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',  # Use 'sparse_categorical_crossentropy' for integer labels
                  metrics=['accuracy'])

    history = model.fit(X_train, y_train, epochs=20, validation_split=0.2, batch_size=32)

    test_loss, test_acc = model.evaluate(X_test, y_test)
    print(f'Test accuracy: {test_acc}')

    return model, history

In [326]:
dnn(X, y)

Epoch 1/20
21/21 [==============================] - 0s 4ms/step - loss: 1.3438 - accuracy: 0.4277 - val_loss: 1.2390 - val_accuracy: 0.4658
Epoch 2/20
21/21 [==============================] - 0s 2ms/step - loss: 1.1470 - accuracy: 0.5459 - val_loss: 1.1610 - val_accuracy: 0.5280
Epoch 3/20
21/21 [==============================] - 0s 1ms/step - loss: 1.0311 - accuracy: 0.5708 - val_loss: 1.1998 - val_accuracy: 0.4783
Epoch 4/20
21/21 [==============================] - 0s 1ms/step - loss: 0.9737 - accuracy: 0.6034 - val_loss: 1.2507 - val_accuracy: 0.4720
Epoch 5/20
21/21 [==============================] - 0s 1ms/step - loss: 1.0090 - accuracy: 0.6128 - val_loss: 1.2126 - val_accuracy: 0.4720
Epoch 6/20
21/21 [==============================] - 0s 1ms/step - loss: 0.9537 - accuracy: 0.6019 - val_loss: 1.2285 - val_accuracy: 0.4596
Epoch 7/20
21/21 [==============================] - 0s 1ms/step - loss: 0.9282 - accuracy: 0.6221 - val_loss: 1.2698 - val_accuracy: 0.4720
Epoch 8/20
21/21 [==

(<keras.engine.sequential.Sequential at 0x30bc38710>,
 <keras.callbacks.History at 0x304f4de10>)